# Input Data Creation

This script performs the cohort selection and data extraction for the convolutional neural network. It takes all hospital admissions for patients with heart failure and retreives the departments, as well as the laboratory values and demographic information. This project was conducted with MIMIC-IV 0.4, which is important, as version 1.0 was released just recently.

In [ ]:
import numpy as np
from psycopg2 import connect
import pandas as pd
import pm4py
import numpy as np
import pandasql as ps
from pm4py.objects.conversion.log import converter as log_converter
con = connect(dbname="postgres", host="127.0.0.1", user="postgres", password="1234")
con.set_client_encoding('utf8')
cursor = con.cursor()

In [ ]:
def get_adms(df, hadm_ids):
    cursor.execute('SELECT * FROM mimic_core.admissions where hadm_id = any(%s)', [hadm_ids])
    adms = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    adms = pd.DataFrame(adms, columns=cols)
    b_adms = adms.loc[adms["hadm_id"].isin(hadm_ids)]
    b_adms.drop("subject_id", axis=1, inplace=True)
    b_adms = df.merge(b_adms, on="hadm_id", how="inner")
    b_adms = b_adms.drop_duplicates("hadm_id")
    return b_adms

In [ ]:
#requires get_adms for admission/discharge location!
def get_transfers(df, hadm_ids):
    cursor.execute('SELECT * FROM mimic_core.transfers where hadm_id = any(%s)', [hadms])
    transfers = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    transfers = pd.DataFrame(transfers, columns=cols)
    b_trans = transfers.loc[transfers["hadm_id"].isin(hadm_ids)]
    b_trans = b_trans.sort_values(["subject_id", "hadm_id","intime"])
    b_trans.loc[(b_trans["careunit"].isna()) & (b_trans["eventtype"] == "transfer"), "careunit"] = "Unknown"
    b_trans.loc[(b_trans["careunit"].isna()) & (b_trans["eventtype"] == "admit"), "careunit"] = "Admit"
    b_trans.loc[(b_trans["careunit"].isna()) & (b_trans["eventtype"] == "discharge"), "careunit"] = "Discharge"
        #Set first careunit to admission location
    b_trans = b_trans.drop("subject_id", axis=1)
    b_trans = b_trans.merge(df, on="hadm_id", how="inner")
    b_trans.loc[b_trans["careunit"] == "Discharge", "careunit"] = b_trans["discharge_location"]
    b_trans = b_trans.sort_values(["subject_id", "hadm_id","intime"])
    first_careunit = b_trans.loc[~b_trans.duplicated("hadm_id", keep="first")]
    admission_location = []
    for index, row in first_careunit.iterrows():
        add_row = row
        add_row["careunit"] = row["admission_location"]
        add_row["outtime"] = row["intime"] 
        add_row["transfer_id"] = np.nan
        add_row["intime"] = add_row["outtime"] - pd.Timedelta(seconds=1)
        admission_location.append(add_row)
    admission_location = pd.DataFrame(admission_location)
    b_trans_admission_location = pd.concat([first_careunit, admission_location])
    b_trans = b_trans.loc[b_trans.duplicated("hadm_id", keep="first")]
    b_trans = pd.concat([b_trans, b_trans_admission_location])
    b_trans = b_trans.sort_values(["subject_id","hadm_id", "intime"])
    return b_trans

In [ ]:
#requires get_transfers
def get_patients(df):
    cursor.execute("SELECT * FROM mimic_core.patients")
    patients = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    patients = pd.DataFrame(patients, columns=cols)
    b_trans_patient = df.merge(patients, on="subject_id", how="inner")
    b_trans_patient["transfer_year"] = b_trans_patient.apply(lambda x: x["intime"].year , axis=1)
    b_trans_patient["transfer_age"] = (b_trans_patient["transfer_year"] - b_trans_patient["anchor_year"]) + b_trans_patient["anchor_age"]
    b_trans_patient["anchor_real_year"] = b_trans_patient["anchor_year_group"].str.slice(0,4)
    b_trans_patient["anchor_real_year"] = pd.to_numeric(b_trans_patient["anchor_real_year"])
    b_trans_patient["anchor_real_year"] = b_trans_patient["anchor_real_year"] + 1
    b_trans_patient["transfer_real_year"] = b_trans_patient["anchor_real_year"] + b_trans_patient["transfer_year"] - b_trans_patient["anchor_year"]
    b_trans_patient.loc[b_trans_patient["transfer_real_year"] == 2021, "transfer_real_year"] = 2020
    b_trans_patient = b_trans_patient.sort_values(["hadm_id", "intime"])
    
    #set patient age for hospital admission according to first transfer in admission
    b_trans_patient.loc[~b_trans_patient.duplicated("hadm_id", keep="first"), "admission_age"] = b_trans_patient["transfer_age"]
    b_trans_patient.loc[b_trans_patient["admission_age"] <= 18, "admission_age_group"] = "0-18"
    b_trans_patient.loc[(b_trans_patient["admission_age"] <= 25) & (b_trans_patient["admission_age"] > 18), "admission_age_group"] = "19-25"
    b_trans_patient.loc[(b_trans_patient["admission_age"] <= 35) & (b_trans_patient["admission_age"] > 25), "admission_age_group"] = "26-35"
    b_trans_patient.loc[(b_trans_patient["admission_age"] <= 45) & (b_trans_patient["admission_age"] > 35), "admission_age_group"] = "36-45"
    b_trans_patient.loc[(b_trans_patient["admission_age"] <= 55) & (b_trans_patient["admission_age"] > 45), "admission_age_group"] = "46-55"
    b_trans_patient.loc[(b_trans_patient["admission_age"] <= 65) & (b_trans_patient["admission_age"] > 55), "admission_age_group"] = "56-65"
    b_trans_patient.loc[(b_trans_patient["admission_age"] <= 75) & (b_trans_patient["admission_age"] > 65), "admission_age_group"] = "66-75"
    b_trans_patient.loc[(b_trans_patient["admission_age"] <= 85) & (b_trans_patient["admission_age"] > 75), "admission_age_group"] = "76-85"
    b_trans_patient.loc[(b_trans_patient["admission_age"] > 85), "admission_age_group"] = "85+"
   ###create patient groups!!#####
    return b_trans_patient

In [ ]:
def get_services(df, hadm_ids):
    cursor.execute('SELECT * from mimic_hosp.services where hadm_id = any(%s)', [hadm_ids])
    services = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    services = pd.DataFrame(services, columns=cols)
    b_services = services.loc[services["hadm_id"].isin(hadms)]
    b_services = b_services.drop("subject_id", axis=1)
    b_services = b_services[["hadm_id", "transfertime", "curr_service"]]
    
    sqlcode = '''
    select *
    from df
    left join b_services on df.hadm_id=b_services.hadm_id
    where b_services.transfertime >= df.intime and  b_services.transfertime < df.outtime 

    '''

    newdf = ps.sqldf(sqlcode,locals())
    newdf = newdf.loc[:,~newdf.columns.duplicated()]
    newdf = newdf.drop_duplicates(["hadm_id", "careunit", "intime", "outtime", "curr_service"])
    df = df.reset_index()
    df.drop("index", axis=1, inplace=True)
    for index, row in newdf.iterrows():
        df.loc[(df["hadm_id"] == row["hadm_id"]) & (df["intime"] == row["intime"]) & (df["outtime"] == row["outtime"]), "service"] = row["curr_service"]
        df.loc[(df["hadm_id"] == row["hadm_id"]) & (df["intime"] == row["intime"]) & (df["outtime"] == row["outtime"]), "service_time"] = row["transfertime"]
    return df

In [ ]:
def get_procedures(df, hadm_ids):
    cursor.execute('SELECT * from mimic_hosp.procedures_icd where hadm_id = any(%s)', [hadm_ids])
    proc = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    proc = pd.DataFrame(proc, columns=cols)
    cursor.execute("SELECT * from mimic_hosp.d_icd_procedures")
    proc_d = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    proc_d = pd.DataFrame(proc_d, columns=cols)
    bp_proc = proc.loc[proc["hadm_id"].isin(hadm_ids)]
    bp_proc = bp_proc.merge(proc_d, on=["icd_code", "icd_version"], how="inner")
    d = bp_proc.groupby(['subject_id','hadm_id']).agg({"icd_code":lambda x: list(x), "seq_num":lambda x: list(x), "icd_version":lambda x: list(x), "long_title":lambda x: list(x)})
    d = d.rename(columns={"icd_code":"proc_icd_code", "seq_num":"proc_seq_num", "icd_version":"proc_icd_version", "long_title":"proc_long_title"})
    d = d.reset_index()
    d = d.drop("subject_id", axis=1)
    df = df.merge(d, on="hadm_id", how="left")
    return df


In [ ]:
def get_meds(hadm_ids):
    cursor.execute('select * from mimic_hosp.pharmacy where hadm_id = any(%s)', [hadm_ids])
    pharmacy = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    pharmacy = pd.DataFrame(pharmacy, columns=cols)
    pharmacy = pharmacy.sort_values(["subject_id", "hadm_id", "starttime"])
    pharmacy = pharmacy.loc[~pharmacy["medication"].isna()]
    return pharmacy

In [ ]:
def get_med_count(df, transfers, hadm_ids):
    cursor.execute('select * from mimic_hosp.pharmacy where hadm_id = any(%s)', [hadm_ids])
    pharmacy = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    pharmacy = pd.DataFrame(pharmacy, columns=cols)
    pharmacy = pharmacy.sort_values(["subject_id", "hadm_id", "starttime"])
    pharmacy = pharmacy.loc[~pharmacy["medication"].isna()]
    
    sqlcode = '''
    select *
    from pharmacy
    left join transfers on pharmacy.hadm_id=transfers.hadm_id
    where pharmacy.starttime >= transfers.intime and pharmacy.starttime <= transfers.outtime
    '''

    newdf = ps.sqldf(sqlcode,locals())
    newdf = newdf.loc[:,~newdf.columns.duplicated()]
    med_count = newdf.groupby(["hadm_id","transfer_id"]).count()
    med_count = med_count.reset_index()
    med_count = med_count.drop(['pharmacy_id', 'poe_id',
       'starttime', 'stoptime', 'medication', 'proc_type', 'status',
       'entertime', 'verifiedtime', 'route', 'frequency', 'disp_sched',
       'infusion_type', 'sliding_scale', 'lockout_interval', 'basal_rate',
       'one_hr_max', 'doses_per_24_hrs', 'duration', 'duration_interval',
       'expiration_value', 'expiration_unit', 'expirationdate', 'dispensation',
       'fill_quantity', 'eventtype', 'intime', 'outtime',
       'drg_type', 'drg_code', 'description', 'drg_severity', 'drg_mortality',
       'count_icd', '1_icd', '1_desc_icd', '2_icd', '2_desc_icd', '3_icd',
       '3_desc_icd', 'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'hospital_expire_flag', 'careunit'], axis=1)
    med_count = med_count.rename(columns={"subject_id":"med_count"})
    df = df.merge(med_count, on=["hadm_id", "transfer_id"], how="left")
    df = df.drop_duplicates(["hadm_id", "transfer_id", "med_count"])
    return df

In [ ]:
def get_diagnoses(df, hadm_ids, n):
    cursor.execute('SELECT * FROM mimic_hosp.diagnoses_icd where hadm_id = any(%s)', [hadm_ids])
    icds = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    icds = pd.DataFrame(icds, columns=cols)
    
    cursor.execute("SELECT * FROM mimic_hosp.d_icd_diagnoses")
    desc_icd = cursor.fetchall()
    cols = list(map(lambda x: x[0], cursor.description))
    desc_icd = pd.DataFrame(desc_icd, columns=cols)
    desc_icd = desc_icd[["icd_code", "long_title"]]
    
    b_icds = icds.loc[icds["hadm_id"].isin(hadm_ids)]
    count_icd = b_icds.groupby("hadm_id").count()
    count_icd = count_icd.reset_index()
    count_icd = count_icd[["hadm_id", "seq_num"]]
    df = df.merge(count_icd, on="hadm_id", how="inner").rename(columns={"seq_num":"count_icd"})
    for i in range (1, n+1):
        to_join = b_icds.loc[b_icds["seq_num"] == i][["hadm_id", "icd_code"]]
        df = df.merge(to_join, on="hadm_id", how="left").rename(columns={"icd_code": str(i) + "_icd"})
        df = df.merge(desc_icd, how="left", left_on=(str(i) + "_icd"), right_on="icd_code")
        df = df.rename(columns={"long_title":str(i) + "_desc_icd"})
        df = df.drop("icd_code", axis=1)
    return df

### Cohort Creation

In [ ]:
cursor.execute('SELECT * FROM mimic_hosp.diagnoses_icd')
icds = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
icds = pd.DataFrame(icds, columns=cols)

In [ ]:
pd.set_option("display.max_rows", 1500)
pd.set_option("display.max_columns", 700)

In [ ]:
#consider all icd codes regarding heart failure
hf = icds.loc[icds["icd_code"].str.contains("42821") | (icds["icd_code"].str.contains("42823")) | (icds["icd_code"].str.contains("42831")) |
        (icds["icd_code"].str.contains("42833")) | (icds["icd_code"].str.contains("42841"))| (icds["icd_code"].str.contains("42843"))
        | (icds["icd_code"].str.contains("I5021")) |  (icds["icd_code"].str.contains("I5023")) |(icds["icd_code"].str.contains("I5031"))|
        (icds["icd_code"].str.contains("I5033")) |
        (icds["icd_code"].str.contains("I5041"))|
        (icds["icd_code"].str.contains("I5042"))|
        (icds["icd_code"].str.contains("I5043"))]

In [ ]:
hf = hf.reset_index()
hf = hf.drop("index", axis=1)
hf

In [ ]:
cursor.execute("SELECT * FROM mimic_hosp.d_icd_diagnoses")
desc_icd = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
desc_icd = pd.DataFrame(desc_icd, columns=cols)
desc_icd = desc_icd[["icd_code", "long_title"]]

In [ ]:
hf = hf.merge(desc_icd, on="icd_code", how="inner")

In [ ]:
cursor.execute("SELECT * from mimic_hosp.drgcodes")
drgs = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
drgs = pd.DataFrame(drgs, columns=cols)

In [ ]:
hf_drg = drgs.loc[drgs["hadm_id"].isin(list(hf["hadm_id"]))]
hf_drg = hf_drg.loc[hf_drg["drg_type"] == "APR"].drop_duplicates(["subject_id", "hadm_id", "description"])

In [ ]:
l = list(["Heart Failure", "Cardiac Catheterization w/ Circ Disord Exc Ischemic Heart Disease","Percutaneous Cardiovascular Procedures w/o AMI",
"Cardiac Arrhythmia & Conduction Disorders",
"Acute Myocardial Infarction",
"Percutaneous Cardiovascular Procedures w/ AMI",
"Cardiac Catheterization for Ischemic Heart Disease",
"Cardiac Defibrillator & Heart Assist Anomaly",
"Cardiac Valve Procedures w/ Cardiac Catheterization",
"Coronary Bypass w/ Cardiac Cath Or Percutaneous Cardiac Procedure",
"Other Circulatory System Diagnoses"
         ])

In [ ]:
hf_filter = hf_drg.loc[hf_drg["description"].isin(l)]
hf_filter = hf_filter.sort_values(["hadm_id", "drg_code"])
hf_filter = hf_filter.drop_duplicates("hadm_id", keep="first")
hf_filter = hf_filter.reset_index()
hf_filter.drop("index", axis=1,inplace=True)
hadms = list(hf_filter["hadm_id"])

## Data fetching for the cohort

In [ ]:
cursor.execute('SELECT * FROM mimic_hosp.diagnoses_icd where hadm_id = any(%s)', [hadms])
icds = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
icds = pd.DataFrame(icds, columns=cols)
hf_diag = get_diagnoses(hf_filter, hadms, 3)
hf_adm = get_adms(hf_diag, hadms)
hf_adm = hf_adm.reset_index()
hf_adm.drop("index", axis=1, inplace=True)
hf_t = get_transfers(hf_adm, hadms)
hf_p = get_patients(hf_t)
hf_s = get_services(hf_p, hadms)
hf_proc = get_procedures(hf_s, hadms)
hf_med = get_med_count(hf_proc,hf_t, hadms)

### Laboratory Values

In [ ]:
cursor.execute('SELECT * FROM mimic_hosp.d_labitems')
lab_d = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
lab_d = pd.DataFrame(lab_d, columns=cols)   

In [ ]:
cursor.execute('SELECT * FROM mimic_hosp.labevents where hadm_id = any(%s)', [hadms])
labs = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
labs = pd.DataFrame(labs, columns=cols)

In [ ]:
lab_w_detail = labs.merge(lab_d, on="itemid", how="inner")
lab_w_detail

In [ ]:
hf_med["LOS"] = hf_med["dischtime"] - hf_med["admittime"]
hf_med["transfer_duration"] = hf_med["outtime"] - hf_med["intime"]
lab_w_detail.groupby(["itemid", "label"]).count().sort_values("subject_id", ascending=False)
lab_w_detail = lab_w_detail.sort_values(["hadm_id", "charttime"])
lab_w_detail = lab_w_detail.loc[lab_w_detail["hadm_id"].isin(hadms)]
hf_t = hf_t.loc[hf_t["1_desc_icd"].str.contains("heart failure", na=False)]
hf_t = hf_t.reset_index()
hadms = list(hf_t["hadm_id"].unique())
hadms = list(map(int, hadms))

In [ ]:
sqlcode = '''
select *
from lab_w_detail
left join hf_t on lab_w_detail.hadm_id=hf_t.hadm_id
where lab_w_detail.charttime >= hf_t.intime and lab_w_detail.charttime <= hf_t.outtime
'''

newdf = ps.sqldf(sqlcode,locals())
newdf = newdf.loc[:,~newdf.columns.duplicated()]
med_count = newdf.groupby(["hadm_id","transfer_id"]).count()
med_count = med_count.reset_index()


In [ ]:
x = med_count.drop(['subject_id', 'specimen_id','itemid', 'charttime', 'storetime', 'value', 'valuenum', 'valueuom',
       'ref_range_lower', 'ref_range_upper', 'flag', 'priority', 'comments',
       'label', 'fluid', 'category', 'loinc_code', 'eventtype', 'careunit',
       'intime', 'outtime', 'drg_type', 'drg_code', 'description',
       'drg_severity', 'drg_mortality', 'count_icd', '1_icd', '1_desc_icd',
       '2_icd', '2_desc_icd', '3_icd', '3_desc_icd',
       'admittime', 'dischtime', 'deathtime', 'admission_type',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'marital_status', 'ethnicity', 'edregtime', 'edouttime',
       'hospital_expire_flag'], axis=1)
x = x.rename(columns={"labevent_id":"lab_count"})


In [ ]:
df = hf_med.merge(x, on=["hadm_id", "transfer_id"], how="left")
df = df.drop_duplicates(["hadm_id", "transfer_id", "lab_count"])
df = df.rename({"case:concept:name":"hadm_id"}, axis=1)
df = df.rename({"careunit":"concept:name"}, axis=1)
df.columns

In [ ]:
df = df.drop(['drg_type', 'drg_code', 'description', 'drg_severity',
       'drg_mortality','1_icd', '1_desc_icd', '2_icd',
       '2_desc_icd', '3_icd', '3_desc_icd', 'language','edregtime', 'edouttime','hospital_expire_flag','admission_age_group', 'service', 'service_time', 'proc_icd_code',
       'proc_seq_num', 'proc_icd_version', 'proc_long_title'], axis=1)

In [ ]:
df = df.drop(['anchor_age',
       'anchor_year', 'anchor_year_group', 'dod', 'transfer_year','dod'], axis=1)

In [ ]:
newdf = pd.DataFrame(data=None, columns=df.columns)
d = {'subject_id':'first', 'hadm_id':'first', 'transfer_id':'first', 'eventtype':'first', 
     'anchor_real_year':'first', 'transfer_real_year':'first', 'admission_age':'first', 'med_count':'sum',
       'LOS':'first', 'transfer_duration':'sum', 'lab_count':'sum',    
     'intime':'min', 'outtime':'max', 'count_icd':'first',
    'admittime':'first', 'dischtime':'first', 'deathtime':'first', 'admission_type':'first', 'admission_location':'first', 'discharge_location':'first',
       'insurance':'first', 'marital_status':'first', 'ethnicity':'first', 'gender':'first', 'transfer_age':'first'}  

In [ ]:
df["transfer_duration"] = pd.to_timedelta(df["transfer_duration"])
arr = []

In [ ]:
for i in list(df["hadm_id"].unique()):
    print(i)
    hadm = df.loc[df["hadm_id"] == i]
    consecutive_array = (hadm["concept:name"] != hadm["concept:name"].shift()).cumsum().values
    new_hadm = hadm.groupby([consecutive_array, 'concept:name']).agg(d).reset_index(level=1)
    arr.append(new_hadm)

In [ ]:
newdf = pd.concat(arr, axis=0)
newdf = newdf.drop(["anchor_real_year", "transfer_real_year", "admission_age"], axis=1)
newdf = newdf.reset_index().drop("index", axis=1)
newdf["LOS"] = pd.to_timedelta(newdf["LOS"])
newdf["hadm_id"] = newdf["hadm_id"].astype("object")
hadms = list(newdf["hadm_id"].unique())

#retrieve lab values for the modified list of patients
cursor.execute('SELECT * FROM mimic_hosp.d_labitems')
lab_d = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
lab_d = pd.DataFrame(lab_d, columns=cols)   

cursor.execute('SELECT * FROM mimic_hosp.labevents where hadm_id = any(%s)', [hadms])
hf_lab = cursor.fetchall()
cols = list(map(lambda x: x[0], cursor.description))
hf_lab = pd.DataFrame(hf_lab, columns=cols)

In [ ]:
hf_lab = hf_lab.merge(lab_d, on="itemid", how="inner")
labs_rnn = ["Creatinine", "Urea Nitrogen", "Hemoglobin", "Glucose", "Red Blood Cells"]
hf_lab_new = hf_lab.loc[hf_lab["label"].isin(labs_rnn)]

In [ ]:
#map lab event to department
sqlcode = '''
select *
from hf_lab_new
inner join newdf on hf_lab_new.hadm_id=newdf.hadm_id
where hf_lab_new.charttime >= newdf.intime and hf_lab_new.charttime <= newdf.outtime 

'''

newdf_labs = ps.sqldf(sqlcode,locals())
newdf_labs = newdf_labs.loc[:,~newdf_labs.columns.duplicated()]
newdf_labs = newdf_labs.sort_values(["subject_id", "hadm_id", "charttime"])
newdf_labs = newdf_labs.reset_index()
newdf_labs = newdf_labs.drop("index", axis=1)

In [ ]:
discharge_info = newdf_labs.groupby(["subject_id", "hadm_id"]).last().reset_index()[["subject_id", "hadm_id", "charttime"]]
newdf_labs_2 = newdf_labs
times = list(discharge_info["charttime"])
newdf_labs_2 = newdf_labs_2.loc[newdf_labs_2["charttime"].isin(times)]
newdf_labs_back = newdf_labs
discharge_temp = []

for index, row in discharge_info.iterrows():
    print(index)
    labs = newdf_labs_2.loc[(newdf_labs_2["hadm_id"] == row["hadm_id"]) & (newdf_labs_2["charttime"] == row["charttime"])]
    labs["concept:name"] = "Discharged"
    discharge_temp.append(labs)

In [ ]:
discharge_temp = pd.concat(discharge_temp, axis=0)
labs_disch = pd.concat([newdf_labs_back,discharge_temp])
lab_df = labs_disch.groupby(["subject_id","hadm_id", "concept:name", "label", "intime", "outtime"]).agg({"valuenum":"mean"}).reset_index()
newdf.loc[newdf["eventtype"] == "discharge", "concept:name"] = "Discharged"
hadm_w_discharge = list(newdf.loc[newdf["concept:name"] == "Discharged"]["hadm_id"].unique())
lab_df = lab_df.loc[lab_df["hadm_id"].isin(hadm_w_discharge)]
newdf = newdf.loc[newdf["hadm_id"].isin(hadm_w_discharge)]

lab_df = lab_df.reset_index()
lab_df.drop("index", axis=1, inplace=True)

lab_df = lab_df.reset_index()
lab_df.drop("index", axis=1, inplace=True)

newdf = newdf.reset_index()
newdf.drop("index", axis=1, inplace=True)

newdf["intime"] = newdf["intime"].apply(lambda x: pd.to_datetime(x))
newdf["outtime"] = newdf["outtime"].apply(lambda x: pd.to_datetime(x))
newdf.loc[newdf["outtime"].isna(), "outtime"] = newdf["intime"] + pd.Timedelta(seconds=1)
newdf["transfer_duration"] = newdf["outtime"] - newdf["intime"]

In [ ]:
#lab values of Discharged do not have the correct timestamp yet and cannot be merged!
deps = newdf.loc[newdf["concept:name"] == "Discharged"][["subject_id", "hadm_id", "intime", "outtime", "concept:name"]]
y = lab_df.loc[lab_df["concept:name"] == "Discharged"]
for index, row in y.iterrows():
    print(index)
    time = deps.loc[deps["hadm_id"] == row["hadm_id"]]
    time = time.reset_index()
    lab_df.loc[index, "intime"] = time["intime"][0]
    lab_df.loc[index, "outtime"] = time["outtime"][0]
        

In [ ]:
lab_df["intime"] = lab_df["intime"].apply(lambda x: pd.to_datetime(x))
lab_df["outtime"] = lab_df["outtime"].apply(lambda x: pd.to_datetime(x))
lab_df = lab_df[["subject_id", "hadm_id", "intime", "outtime", "concept:name", "valuenum", "label"]]

lab_pm = newdf
for label in labs_rnn:
    print(label)
    df_single = lab_df.loc[lab_df["label"] == label]
    df_single.rename({"valuenum":label}, axis=1, inplace=True)
    df_single.drop("label", axis=1, inplace=True)
    df_single = df_single[["subject_id", "hadm_id", "intime", "outtime", "concept:name", label]]
    lab_pm = lab_pm.merge(df_single, on=["subject_id", "hadm_id", "intime", "outtime", "concept:name"], how="left")

In [ ]:
lab_pm = lab_pm.loc[~lab_pm["transfer_id"].isna()]
disch_fac = ["SKILLED NURSING FACILITY", "HOME", "HOME HEALTH CARE"]
filtered_pm = lab_pm.loc[lab_pm["discharge_location"].isin(disch_fac)]
filtered_pm = filtered_pm.reset_index().drop("index", axis=1)
filtered_pm = filtered_pm.drop("deathtime", axis=1)
filtered_pm = filtered_pm.drop(['transfer_id', 'eventtype'], axis=1)
filtered_pm.to_csv("../data/AI_HEART_FAILURE_CNN.csv")